In [1]:
import torch
import seaborn as sns
import pandas as pd
import transformers
import evaluate
from transformers import pipeline, AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, load_from_disk
from collections import Counter, defaultdict
from transformers import DataCollatorWithPadding
from pathlib import Path
import numpy as np
from sklearn.metrics import f1_score, classification_report, accuracy_score
from tqdm.auto import tqdm
import re
from nltk.tokenize import sent_tokenize
from sklearn.linear_model import LinearRegression
sns.set()

In [23]:
#dataset = load_from_disk('/datadrive_2/')
#test_data = dataset['test']

cache_dir = '/datadrive_2/hf_cache/'
dataset = load_from_disk("/datadrive_2/HMD_chunked_100_test/")
dataset #= dataset['train']

Dataset({
    features: ['year', 'nlp', 'pol', 'loc', 'sentences', 'ocr'],
    num_rows: 11315511
})

## Classify by Political Leaning

In [4]:
pol_pattern = re.compile(r'\bliberal|\bconservat|\bministers?|\btory\b|\btories\b|\bgovernments?|\bpolitic',re.I)

In [5]:
pol_pattern.findall('liberal governments do not fire their political ministers minister')

['liberal', 'governments', 'politic', 'ministers', 'minister']

In [6]:
# def sent_split(x):
#      return {'data': [
#                 {'sentence':s.lower(),
#                  'length': len(s.split()),
#                  'pol': p, 'loc':l, 'year':y, 'ocr':o,'nlp':n} 
#                      for y,p,l,o,n,t in zip(x['year'],x['pol'],x['loc'],x['ocr_quality_mean'],x['nlp'],x['text']) 
#                       for s in sent_tokenize(t) 
#                          if pol_pattern.findall(s)
#                  ]
#             }

# test_data = dataset.map(sent_split,batched=True, remove_columns=dataset.column_names)

Loading cached processed dataset at /datadrive_2/HMD_context/train/cache-ba07e195c8ec9d10.arrow


In [26]:
test_data = dataset.map(lambda x: {'sentences': x['sentences'].lower()}, num_proc=6).shuffle(seed=42).select(range(15000))

Loading cached processed dataset at /datadrive_2/HMD_chunked_100_test/cache-2d79338e1133aae6.arrow
Loading cached processed dataset at /datadrive_2/HMD_chunked_100_test/cache-6f3d8b9a1021878e.arrow


Loading cached processed dataset at /datadrive_2/HMD_chunked_100_test/cache-b3061892df3a5bbe.arrow


Loading cached processed dataset at /datadrive_2/HMD_chunked_100_test/cache-24d2a5525e368df0.arrow


Loading cached processed dataset at /datadrive_2/HMD_chunked_100_test/cache-06bf12bcec6754fc.arrow
Loading cached processed dataset at /datadrive_2/HMD_chunked_100_test/cache-a090791e087fa494.arrow
Loading cached shuffled indices for dataset at /datadrive_2/HMD_chunked_100_test/cache-2e3366e450e9f2f2.arrow


In [27]:
test_data[10]

{'year': 1853,
 'nlp': 2194,
 'pol': '[lib]',
 'loc': '[london]',
 'sentences': 'a slight natural yearning for theadmirable status quo of our constitution ? doesnot lord john feel that affection for the presenthouse of commons, with all its faults, whichprompts a natural parent to resent the interferenceof a neighbour in the education and even the ne-cessary correction of his offspring ? no father, nomother, ever yet tolerated the hand of thestranger on her child. casually, and by the way,lord john says it—but backhanders are alwaysthe strongest blows—" it would be difficult, per-haps, to find a time in the history of any countrywhen so many measures of importance have beenobtained, without convulsion and',
 'ocr': 0.9639}

In [9]:
#data = test_data.filter(lambda x: x['data.length'] > 25).shuffle(seed=42).select(range(15000))

Loading cached processed dataset at /datadrive_2/HMD_context/train/cache-a30da7bf93707de5.arrow
Loading cached shuffled indices for dataset at /datadrive_2/HMD_context/train/cache-ef1d6e941b6c7082.arrow


In [30]:
def pred_data(example,add_field='year'):
    return {'st_year_sep': f'[{example[add_field]}]' + ' [SEP] ' + example['sentences'] ,
     'year_sep': str(example[add_field]) + ' [SEP] ' + example['sentences'] ,
     'year_date': str(example[add_field]) + ' [DATE] ' + example['sentences'],
        
    }
    
data = test_data.map(pred_data , num_proc=6)

#0:   0%|          | 0/2500 [00:00<?, ?ex/s]

#1:   0%|          | 0/2500 [00:00<?, ?ex/s]

#3:   0%|          | 0/2500 [00:00<?, ?ex/s]

#2:   0%|          | 0/2500 [00:00<?, ?ex/s]

#4:   0%|          | 0/2500 [00:00<?, ?ex/s]

#5:   0%|          | 0/2500 [00:00<?, ?ex/s]

In [31]:
len(data)

15000

In [33]:
lab2code = {'[con]':0,'[lib]':1,'[rad]':2,'[neutr]':3,'[none]':4}
num_labels = len(lab2code)
data = data.map(lambda x: {'label': lab2code[x['pol']]})

  0%|          | 0/15000 [00:00<?, ?ex/s]

In [34]:
data[0]

{'year': 1859,
 'nlp': 2084,
 'pol': '[neutr]',
 'loc': '[liverpool]',
 'sentences': 'sitenirranv, roy institute as tou ry, v. ireno. i.—nervous debility, loss of kient.„2,l:oo!op orof sight, prostration of physical energy, 1;\': asor\'\'business, study, or society, with dr. marston \'01coax. men.o• heiti 0addressed specially to young 14. ...e oh 0%—marrlagerbanodbilgaltetipoltnsodanudetilrn°ooimp ten &c. \' addressed to those who dam. 1010 . totof removing all impediments. qi thee dirpthaenß:orrrs4 2ar.cll\'of healthful children, clearly expl.-11:91p .no. 3.—tiie great social evil. 11\' wee voerdiseases which result from it, with da. minor ~,fc „i;,da. massrosi continues to be consulted pe ,e,:.„penned with.veoftiit.reatment, by which msactinr luso* a\'\'\'s,y.o,,tifi,,:vilti9.residence,ez ens-......r5z 0 d-6 orby letter,47, eitherin at thexrßotal.lnsroittitatetßa: 1171,01\'"raftpaordrticeuiveargry.inf°""ti°l4 70-00.',
 'ocr': 0.643,
 'st_year_sep': '[1859] [SEP] sitenirranv, roy inst

In [35]:
test_size = int(len(data)*.2)
train_test = data.train_test_split(test_size=test_size, seed=1984)
test_set = train_test['test']
val_size = int(len(train_test['train'])*.15)
train_val =  train_test['train'].train_test_split(test_size=val_size, seed=1984)

In [36]:
train_val

DatasetDict({
    train: Dataset({
        features: ['year', 'nlp', 'pol', 'loc', 'sentences', 'ocr', 'st_year_sep', 'year_sep', 'year_date', 'label'],
        num_rows: 10200
    })
    test: Dataset({
        features: ['year', 'nlp', 'pol', 'loc', 'sentences', 'ocr', 'st_year_sep', 'year_sep', 'year_date', 'label'],
        num_rows: 1800
    })
})

In [37]:
checkpoints = [('distilbert','distilbert-base-uncased','[SEP]','year_sep'),
               ('hmd_distilbert','/datadrive_2/bnert-hmd','[SEP]','year_sep'),
               ('bnert-time-st-y','/datadrive_2/bnert-time-st-y','[SEP]','st_year_sep'),
               ('bnert-time-y','/datadrive_2/bnert-time-y','[DATE]','year_date'),
               ('bnert-time-y_masked_25','/datadrive_2/bnert-time-y_masked_25','[DATE]','year_date'),
               ('bnert-time-y_masked_75','/datadrive_2/bnert-time-y_masked_75','[DATE]','year_date'),
               ('bnert-pol-st','/datadrive_2/bnert-pol-st','[SEP]','year_sep'),
               ('bnert-pol','/datadrive_2/bnert-pol','[SEP]','year_sep')]

model_dict = defaultdict(dict)
for name,checkpoint, st, sent_col in checkpoints:
    model_dict[name]['model'] = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=num_labels)
    model_dict[name]['tokenizer'] = AutoTokenizer.from_pretrained(checkpoint)
    #model_dict[name]['special_token'] = st
    model_dict[name]['sentences'] = sent_col

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /home/kbeelen/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weig

loading weights file /datadrive_2/bnert-time-st-y/pytorch_model.bin
Some weights of the model checkpoint at /datadrive_2/bnert-time-st-y were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at /datadrive_2/bnert-time-st-y and a

loading file /datadrive_2/bnert-time-y_masked_75/vocab.txt
loading file /datadrive_2/bnert-time-y_masked_75/tokenizer.json
loading file /datadrive_2/bnert-time-y_masked_75/added_tokens.json
loading file /datadrive_2/bnert-time-y_masked_75/special_tokens_map.json
loading file /datadrive_2/bnert-time-y_masked_75/tokenizer_config.json
loading configuration file /datadrive_2/bnert-pol-st/config.json
Model config DistilBertConfig {
  "_name_or_path": "/datadrive_2/bnert-pol-st",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "p

In [39]:
train_val = train_val.remove_columns(['nlp', 'ocr', 'loc'])
train_val

DatasetDict({
    train: Dataset({
        features: ['year', 'pol', 'sentences', 'st_year_sep', 'year_sep', 'year_date', 'label'],
        num_rows: 10200
    })
    test: Dataset({
        features: ['year', 'pol', 'sentences', 'st_year_sep', 'year_sep', 'year_date', 'label'],
        num_rows: 1800
    })
})

In [40]:
#def add_text_col(example,source):
#    return {'text' : example[source]}

def preprocess_function(examples, target_col):
    return tokenizer(examples[target_col], truncation=True )

In [41]:
result_dict = defaultdict(dict)

for name, mdict in model_dict.items():
    print(f'Creating a model for {name}')
    tokenizer = model_dict[name]['tokenizer']
    model = model_dict[name]['model']
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    sent_col = model_dict[name]['sentences']
    
    #train_val = train_val.map(add_text_col,fn_kwargs={'source': sent_col})
    train_val = train_val.map(preprocess_function,fn_kwargs={'target_col': sent_col})
    
    training_args = TrainingArguments(
    seed=1984,
    output_dir=f"./results_{name}",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
        )

    trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_val["train"],
    eval_dataset=train_val["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
        )


    trainer.train()
    
    model.save_pretrained(f'/datadrive_2/{name}-pol')
    tokenizer.save_pretrained(f"/datadrive_2/{name}-pol")
    
    test_set = test_set.map(preprocess_function,fn_kwargs={'target_col': sent_col})
    predictions = trainer.predict(test_set)
    preds = np.argmax(predictions.predictions, axis=-1)
    result_dict[name]['f1_macro'] = f1_score(preds,predictions.label_ids,average='macro')
    result_dict[name]['f1_micro'] = f1_score(preds,predictions.label_ids,average='micro')
    result_dict[name]['accuracy']  = accuracy_score(preds,predictions.label_ids)

Creating a model for distilbert


  0%|          | 0/10200 [00:00<?, ?ex/s]

  0%|          | 0/1800 [00:00<?, ?ex/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: pol, sentences, year_sep, year_date, st_year_sep, year. If pol, sentences, year_sep, year_date, st_year_sep, year are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/datadrive_2/lm2/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training

Step,Training Loss
500,0.661400
1000,0.529900
1500,0.445600
2000,0.366100
2500,0.274700
3000,0.211800


...

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: pol, sentences, year_sep, year_date, st_year_sep, year. If pol, sentences, year_sep, year_date, st_year_sep, year are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/datadrive_2/lm2/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training

Step,Training Loss
500,0.652400
1000,0.443400
1500,0.363700
2000,0.280400
2500,0.212300
3000,0.159700


Saving model checkpoint to ./results_bnert-pol/checkpoint-500
Configuration saved in ./results_bnert-pol/checkpoint-500/config.json
Model weights saved in ./results_bnert-pol/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results_bnert-pol/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results_bnert-pol/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./results_bnert-pol/checkpoint-1000
Configuration saved in ./results_bnert-pol/checkpoint-1000/config.json
Model weights saved in ./results_bnert-pol/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results_bnert-pol/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results_bnert-pol/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./results_bnert-pol/checkpoint-1500
Configuration saved in ./results_bnert-pol/checkpoint-1500/config.json
Model weights saved in ./results_bnert-pol/checkpoint-1500/pytorch_model.bin
tokenizer config

  0%|          | 0/3000 [00:00<?, ?ex/s]

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: pol, sentences, year_sep, ocr, loc, st_year_sep, year_date, year, nlp. If pol, sentences, year_sep, ocr, loc, st_year_sep, year_date, year, nlp are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3000
  Batch size = 16


In [42]:
results_df = pd.DataFrame.from_dict(result_dict, orient='index')

In [43]:
print(results_df.round(3).to_latex())

\begin{tabular}{lrrr}
\toprule
{} &  f1\_macro &  f1\_micro &  accuracy \\
\midrule
distilbert             &     0.631 &     0.800 &     0.800 \\
hmd\_distilbert         &     0.663 &     0.828 &     0.828 \\
bnert-time-st-y        &     0.670 &     0.820 &     0.820 \\
bnert-time-y           &     0.667 &     0.830 &     0.830 \\
bnert-time-y\_masked\_25 &     0.671 &     0.831 &     0.831 \\
bnert-time-y\_masked\_75 &     0.672 &     0.831 &     0.831 \\
bnert-pol-st           &     0.668 &     0.827 &     0.827 \\
bnert-pol              &     0.662 &     0.825 &     0.825 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_397981/2842510611.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results_df.round(3).to_latex())


In [22]:
results_df.to_csv('tables/classsify_pol_regex')